In [11]:
from PIL import Image,ImageDraw
import numpy as np
# file_name = '0000'
file_name = '000000'
parse = Image.open(f'./sample_data/upper_body/{file_name}_4.png')
img = Image.open(f'./sample_data/upper_body/{file_name}_0.jpg')

# parse = Image.open(f'./check_data/{file_name}_parse.png')
# img = Image.open(f'./check_data/{file_name}_img.jpeg')
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
parse_array = np.array(parse)
parse_array.shape

(1024, 768)

In [12]:
import json
with open(f"./sample_data/upper_body/{file_name}_2.json",'r') as f:
# with open(f"./check_data/{file_name}_2.json",'r') as f:
    pose_label = json.load(f)
    pose_data = pose_label['keypoints']
    pose_data = np.array(pose_data)
    pose_data = pose_data[:, :2]*2
pose_data

array([[352.,   0.],
       [350., 120.],
       [262., 120.],
       [244., 280.],
       [244., 418.],
       [438., 120.],
       [462., 274.],
       [462., 414.],
       [294., 422.],
       [292., 636.],
       [302., 816.],
       [404., 418.],
       [402., 630.],
       [386., 816.],
       [330.,   0.],
       [372.,   0.],
       [306.,   0.],
       [402.,   0.]])

In [4]:
from PIL import Image
import numpy as np
original_parse = Image.open('sample_data/upper_body/000000_5.png')
np_parse_original = np.array(original_parse)
print(np_parse_original.shape)

(1024, 768)


In [11]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [13]:
parse_head = ((parse_array == 1).astype(np.float32) +
                  (parse_array == 2).astype(np.float32)+
                  (parse_array == 11).astype(np.float32)+
                  (parse_array == 3).astype(np.float32)
                  )
parse_upper = (
            (parse_array == 4).astype(np.float32) +
            (parse_array == 14).astype(np.float32) +
            (parse_array == 15).astype(np.float32) 
            )
parse_lower = (
                   (parse_array == 5).astype(np.float32) +
                   (parse_array == 6).astype(np.float32) +
            (parse_array == 8).astype(np.float32) +
                   (parse_array == 12).astype(np.float32) +
                   (parse_array == 13).astype(np.float32) )
parse_hand = ((np_parse_original == 3).astype(np.float32) +
                (np_parse_original == 4).astype(np.float32))

In [14]:
r = 20
img = np.array(img)
img[parse_lower>0,:] = 0
img = Image.fromarray(img)
agnostic = img.copy()

# agnostic.show()

agnostic_draw = ImageDraw.Draw(agnostic)
length_a = np.linalg.norm(pose_data[5] - pose_data[2]+1e-8)
length_b = np.linalg.norm(pose_data[11] - pose_data[8]+1e-8)
point = (pose_data[8] + pose_data[11]) / 2
pose_data[8] = point + (pose_data[8] - point) / length_b * length_a
pose_data[11] = point + (pose_data[11] - point) / length_b * length_a

# mask legs
agnostic_draw.line([tuple(pose_data[i]) for i in [8, 11]], 'black', width=r*5)
# agnostic.show()

for i in [9, 12]:
# for i in [8, 9, 11, 12]:
    pointx, pointy = pose_data[i]
    agnostic_draw.ellipse((pointx-r*3, pointy-r*3, pointx+r*3, pointy+r*3), 'black', 'black')
# agnostic.show()
for i in [9,10,12,13]:
    if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
        continue
    agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'black', width=r*5)
    
    pointx, pointy = pose_data[i]
    if i in [10,13]:
        pass#agnostic_draw.ellipse((pointx-r, pointy-r, pointx+r, pointy+r), 'black', 'black')
    else:
        agnostic_draw.ellipse((pointx-r*2, pointy-r*2, pointx+r*2, pointy+r*2), 'black', 'black')

agnostic.paste(img, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
agnostic.paste(img, None, Image.fromarray(np.uint8(parse_upper * 255), 'L'))
agnostic.paste(img, None, Image.fromarray(np.uint8(parse_hand * 255), 'L'))

agnostic.show()
agnostic.save("./check_fig.png")